In [1]:
import keras
import numpy as np
import pickle
import os
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.initializers import he_normal
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras import backend as K

Using TensorFlow backend.


In [2]:
#-----data dir----
data_dir = "./data"
#-----------------

In [3]:
def scheduler(epoch):
  learning_rate_init = 0.001
  if epoch > 55:
    learning_rate_init = 0.0002
  if epoch > 70:
    learning_rate_init = 0.00005
  return learning_rate_init

def unpickle(filename):
  file = os.path.join(data_dir, filename)
  with open(file, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')
  return dict

class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    # customize your behavior
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 13:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 23:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 33:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)


In [4]:
#-------- dimensions ---------
height, width = 32, 32
channel = 3
if K.image_data_format() == 'channels_first':
    input_shape = (channel, height, width)
else:
    input_shape = (height, width, channel)
#-----------------------------

In [5]:
train_size = 50000
test_size = 10000

#--- coarse 1 classes ---
coarse1_classes = 8
#--- coarse 2 classes ---
coarse2_classes = 20
#--- fine classes ---
num_classes  = 100

batch_size   = 128
epochs       = 80

In [6]:
#--- file paths ---
log_filepath = './tb_log_vgg16_hierarchy_dynamic/'
weights_store_filepath = './vgg16_weights_hierarchy_dynamic/'
retrain_id = '101'
model_name = 'weights_vgg16_dynamic_cifar_100_'+retrain_id+'.h5'
model_path = os.path.join(weights_store_filepath, model_name)

In [7]:
#----------get VGG16 pre-trained weights--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')

553467904/553467096 [==============================] - 525s 1us/step


In [9]:
#---------get data---------
meta = unpickle("meta")
test = unpickle("test")
train = unpickle("train")

In [10]:
#-------------------- data loading ----------------------
x_train = np.reshape(train[b'data'], (train_size, channel, height, width)).transpose(0, 2, 3, 1).astype("float32")
x_train = (x_train-np.mean(x_train)) / np.std(x_train)

x_test = np.reshape(test[b'data'], (test_size, channel, height, width)).transpose(0, 2, 3, 1).astype("float32")
x_test = (x_test-np.mean(x_test)) / np.std(x_test)

y_train = np.zeros((train_size, num_classes)).astype('float32')
y_c2_train = np.zeros((train_size, coarse2_classes)).astype('float32')

y_test = np.zeros((test_size, num_classes)).astype('float32')
y_c2_test = np.zeros((test_size, coarse2_classes)).astype('float32')

y_train[np.arange(train_size), train[b'fine_labels']] = 1
y_c2_train[np.arange(train_size), train[b'coarse_labels']] = 1

y_test[np.arange(test_size), test[b'fine_labels']] = 1
y_c2_test[np.arange(test_size), test[b'coarse_labels']] = 1

c2_to_f = np.zeros((coarse2_classes, num_classes)).astype('float32')
fine_unique, fine_unique_indices = np.unique(train[b'fine_labels'], return_index=True)
for i in fine_unique_indices:
  c2_to_f[train[b'coarse_labels'][i]][train[b'fine_labels'][i]] = 1

parent_c2 = {
  0:0, 1:0, 2:1, 3:2, 
  4:1, 5:2, 6:2, 7:3, 
  8:4, 9:5, 10:5, 11:4, 
  12:4, 13:3, 14:6, 15:4, 
  16:4, 17:1, 18:7, 19:7
}

y_c1_train = np.zeros((y_c2_train.shape[0], coarse1_classes)).astype("float32")
y_c1_test = np.zeros((y_c2_test.shape[0], coarse1_classes)).astype("float32")
for i in range(y_c1_train.shape[0]):
  y_c1_train[i][parent_c2[np.argmax(y_c2_train[i])]] = 1.0
for i in range(y_c1_test.shape[0]):
  y_c1_test[i][parent_c2[np.argmax(y_c2_test[i])]] = 1.0

del(train)
del(test)
#---------------------------

In [12]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)
print("y_c1_train shape: ", y_c1_train.shape)
print("y_c1_test shape: ", y_c1_test.shape)
print("y_c2_train shape: ", y_c2_train.shape)
print("y_c2_test shape: ", y_c2_test.shape)

x_train shape:  (50000, 32, 32, 3)
x_test shape:  (10000, 32, 32, 3)
y_train shape:  (50000, 100)
y_test shape:  (10000, 100)
y_c1_train shape:  (50000, 8)
y_c1_test shape:  (10000, 8)
y_c2_train shape:  (50000, 20)
y_c2_test shape:  (10000, 20)


In [13]:
y_c1_test[0]

array([0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)